In [132]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [133]:
def I(E_r, E_s, S=0, mu=0, plot=False):
    lambda_ = 532e-9  # Longitud de onda en metros
    k = 2 * np.pi / lambda_  # Número de onda
    x = np.linspace(-k, k, 500)  # Rango de posiciones en el eje x
    
    # Cálculo de términos de intensidad
    Omega = np.linalg.norm(E_r)**2 + np.linalg.norm(E_s)**2
    Psi = np.abs(np.dot(E_r.T, E_s))[0, 0]
    
    # Cálculo de la fase relativa vartheta
    numerador = -np.abs(E_r[1]) * np.abs(E_s[1]) * np.sin(np.angle(E_r[1]) - np.angle(E_s[1]))
    denominador = np.abs(E_r[0]) * np.abs(E_s[0]) + np.abs(E_r[1]) * np.abs(E_s[1]) * np.cos(np.angle(E_r[1]) - np.angle(E_s[1]))
    vartheta = np.arctan2(numerador, denominador)
    
    # Modulación de fase
    zeta = k * x - vartheta
    cos_term = np.cos(mu + zeta)
    I = Omega + Psi * cos_term
    info = [Omega, Psi, vartheta]

    # Normalización de la intensidad para escalar entre 0 y 255 (imagen en escala de grises)
    I_norm = (I - I.min()) / (I.max() - I.min()) * 255
    I_norm = np.uint8(I_norm)

    # Crear imagen de patrón de interferencia en escala de grises
    height = 500  # Altura de la imagen en píxeles
    pattern_image = np.tile(I_norm, (height, 1))

    # Aplicar un filtro Gaussiano para suavizar las franjas
    pattern_image = cv2.GaussianBlur(pattern_image, (5, 5), 0)
    
    if plot:
        # Graficar el interferograma
        plt.figure(figsize=(8, 6))
        plt.imshow(pattern_image, cmap='gray', aspect='auto', extent=[x.min(), x.max(), 0, height])
        plt.xlabel("x")
        plt.ylabel("Intensidad")
        plt.title("Interferograma")
        #plt.colorbar(label='Intensidad Normalizada')
        plt.show()
    
    return info ,I


In [187]:
Er_1 = np.array([[1], [0]])  
Er_2 = np.array([[1], [1]])  
Es = np.array([[1], [2*np.exp(1j * np.pi/4)]])

info_1, i_1 = I(Er_1,Es)
info_2, i_2 = I(Er_2,Es)

Esx = info_1[1]/np.abs(Er_1[0])
Esy = np.sqrt(info_1[0] - Esx**2 - Er_1[0]**2)

Es_c = np.array([[Esx], [Esy]])  

# Calculamos el numerador y denominador
numerador = (np.linalg.norm(Er_2)**2 + np.linalg.norm(Es_c)**2) * np.tan(info_2[2])[0]
denominador = np.linalg.norm(Er_2)**2 + np.linalg.norm(Es_c)**2 + 2 * np.abs(Er_2[0])[0] * np.abs(Es_c[0])[0] * np.abs(Er_2[1])[0] * np.abs(Es_c[1])[0]



# Calculamos la fase usando atan2
#delta_phi_s =  np.arctan2(numerador,denominador)
delta_phi_s = info_2[2]

print(Esx)
print(Esy)
print(delta_phi_s)
print(np.angle(Es[1]))

[1.]
[2.]
[0.52990279]
[0.78539816]


In [191]:
import sympy as sp

# Definir variables simbólicas
vartheta = info_2[2]  # Definir como variable simbólica si no está definida
phi_s = sp.Symbol('Delta_phi_s')  # Incógnita a despejar

# Definir matrices simbólicas
Es_c = sp.Matrix([[1], [2]])  
Er_2 = sp.Matrix([[1], [1]])  

# Extraer elementos como escalares
Es_c_1 = Es_c[0, 0]
Es_c_2 = Es_c[1, 0]
Er_2_1 = Er_2[0, 0]
Er_2_2 = Er_2[1, 0]

# Expresión corregida con elementos escalares
expr = sp.atan2(-Er_2_2 * Es_c_2 * sp.sin(-phi_s), 
                Er_2_1 * Es_c_1 + Er_2_2 * Es_c_2 * sp.cos(-phi_s)) - vartheta

# Resolver para Δφ_s
solution = sp.solve(expr, phi_s)

# Imprimir solución
print(solution if solution else "No se encontraron soluciones simbólicas.")


[(0.785398163397449,)]


In [ ]:
import sympy as sp

# Definir variables simbólicas
vartheta = sp.Symbol('vartheta')  
phi_s = sp.Symbol('Delta_phi_s')  

# Definir elementos simbólicos de las matrices
Es_c_1, Es_c_2 = sp.symbols('Es_c_1 Es_c_2')
Er_2_1, Er_2_2 = sp.symbols('Er_2_1 Er_2_2')

# Expresión general con variables simbólicas
expr = sp.atan2(-Er_2_2 * Es_c_2 * sp.sin(-phi_s), 
                Er_2_1 * Es_c_1 + Er_2_2 * Es_c_2 * sp.cos(-phi_s)) - vartheta

# Resolver para Δφ_s
solution = sp.solve(expr, phi_s)

# Imprimir solución simbólica
print(solution if solution else "No se encontraron soluciones simbólicas.")


ValueError: 
expected a one-dimensional and numerical function